In [88]:
from joblib import load
from alibi.explainers import CounterFactual
import pandas as pd
import tensorflow as tf
import numpy as np

tf.compat.v1.disable_eager_execution()

In [89]:
clf = load("models/model.joblib")

In [90]:
predict_fn = lambda x: clf.predict_proba(x)

In [91]:
shape = (1,4)
print(shape)

(1, 4)


In [92]:
explainer = CounterFactual(predict_fn, shape=shape, target_class=1, target_proba=0.51, debug=True, write_dir=".", max_iter=300, lam_init=0.01, tol=0.1)

In [93]:
df = np.array([21.0, 0.1, 0.1, 100]).reshape(1, -1)
df

array([[ 21. ,   0.1,   0.1, 100. ]])

In [94]:
print(predict_fn(df))
print(clf.predict(df))

[[0.71 0.29]]
[0]


In [100]:
%timeit -r 5 -n 5 cf = explainer.explain(df)

49.9 s ± 2.7 s per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [99]:
print(cf.data['cf']['X'])
print("diff: ", cf.data['cf']['X'] - df)

[[2.1000000e+01 7.5534493e-02 1.9703722e-01 1.0000000e+02]]
diff:  [[ 0.         -0.02446551  0.09703722  0.        ]]


In [97]:
predict_fn(cf.data['cf']['X'])

array([[0.49, 0.51]])